##1) Optimization problem for designing a box shaped structure with largest possible volume:

In [23]:
import cvxpy as cp

# Given data according to question
a = 0.8
b = 1.2

#defining varaiables
h = cp.Variable(pos=True, name="h")
w = cp.Variable(pos=True, name="w")
d = cp.Variable(pos=True, name="d")

volume = h * w * d
Total_wall_area = 2 * (h * w + h * d)
Total_floor_area = w * d
h_w_ratio = h/w
d_w_ratio = d/w

#defining constraints
constraints = [ Total_wall_area <= 200, Total_floor_area <= 60]
constraints += [ h_w_ratio >= a, h_w_ratio <= b, d_w_ratio >= a, d_w_ratio <= b]

#defining problem
problem = cp.Problem(cp.Maximize(volume), constraints)
assert not problem.is_dcp()
assert problem.is_dgp()
problem.solve(gp=True)

#printing the result
print('Height (in units):%.2g'% h.value)
print('Width (in units):%.2g'% w.value)
print('Depth (in units):%.2g'% d.value)
print('Largest possible volume (in cube units):',problem.value)


Height (in units):6.5
Width (in units):7.7
Depth (in units):7.7
Largest possible volume (in cube units): 387.29833410356673


##2) Power assignment in a wireless communication system:

In [25]:
import numpy as np
def pow_assign(G,max_p,p_rcv,sigma,Group,Group_max,epsilon = 0.001):

  
  n,m = np.shape(G) #where G is the path gain matrix
  delta = np.identity(n)
  S = G*delta # signal power matrix
  I = G-S # interference power matrix
  num_groups = int(np.size(Group,0))

  # normalising the maximum power
  grp_nrm = np.array([group/np.sum(group) for group in Group])
 
  p = cp.Variable((n,1)) #variable p:  power of the n number of transmitters
  best = np.zeros(n)
  # set upper and lower bounds for sub-level set
  u = 1e4
  l = 0

  alpha = cp.Parameter() #alpha= [1/(min SINR)]

  #defining constraints
  constraints = [I@p + sigma <= alpha*S*p, p <= max_p, p >= 0, G*p <= p_rcv, grp_nrm*p <= Group_max]

  # define objective function
  objective = cp.Minimize(alpha)

  #checking if the solution lies between u and l or not
  alpha.value = u

  # Bisection algo:
  maxLoop = int(1e7)
  for i in range(1,maxLoop):
    alpha.value = (u + l)/2.0 #α as the midpoint of the interval
    if u-l <= epsilon:
      break

    #Solving the problem
    prob = cp.Problem(objective, constraints)
    prob.solve()
    
    if prob.status == 'optimal':
      u = alpha.value
      best = p.value
    else:
      l = alpha.value

    if u - l > epsilon and i == (maxLoop-1):
      print("Result did not converged to order epsilon")

  return l,u,alpha.value,best

G = np.array([[1.0,0.1,0.2,0.1,0.0],
                [0.1,1.0,0.1,0.1,0.0],
                [0.2,0.1,2.0,0.2,0.2],
                [0.1,0.1,0.2,1.0,0.1],
                [0.0,0.0,0.2,0.1,1.0]])


n,m = np.shape(G) #here n=m

max_p = np.array([1.]*n).reshape((n,1))
p_rcv = (np.array([5.,5.,5.,5.,5.]).reshape((n,1)))/n #normalised recieved power
sigma = np.array([0.5,0.5,0.5,0.5,0.5]).reshape((n,1))
Group = np.array([[1.,1.,0,0,0],[0,0,1.,1.,1.]])
Group_max = np.array([[2.0],[2.0]]) # max normalised power for groups
l,u,alpha,best= pow_assign(G,max_p,p_rcv,sigma,Group,Group_max)

#printing the results
print('For the given problem the minimum SINR is : %.2g'%(1/alpha))
print('The transmitter powers are :','\n',(best))


For the given problem the minimum SINR is : 0.76
The transmitter powers are : 
 [[0.50736535]
 [0.4845548 ]
 [0.32391971]
 [0.54058924]
 [0.47054301]]
